### LangChain Essentials

# Conversational Memory for Ollama - LangChain #3

Conversational memory allows our chatbots and agents to remember previous interactions within a conversation. Without conversational memory, our chatbots would only ever be able to respond to the last message they received, essentially forgetting all previous messages with each new message.

Naturally, conversations require our chatbots to be able to respond over multiple interactions and refer to previous messages to understand the context of the conversation.

---

> ⚠️ We will be using Ollama for this example allowing us to run everything locally. If you would like to use OpenAI instead, please see the [OpenAI version](https://github.com/aurelio-labs/langchain-course/blob/main/notebooks/openai/03-conversational-memory-openai.ipynb) of this example.

---

---

> ⚠️ If using LangSmith, add your API key below:

In [1]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv(dotenv_path='.env')

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

---

## LangChain's Memory Types

LangChain versions `0.0.x` consisted of various conversational memory types. Most of these are due for deprecation but still hold value in understanding the different approaches that we can take to building conversational memory.

Throughout the notebook we will be referring to these _older_ memory types and then rewriting them using the recommended `RunnableWithMessageHistory` class. We will learn about:

* `ConversationBufferMemory`: the simplest and most intuitive form of conversational memory, keeping track of a conversation without any additional bells and whistles.
* `ConversationBufferWindowMemory`: similar to `ConversationBufferMemory`, but only keeps track of the last `k` messages.
* `ConversationSummaryMemory`: rather than keeping track of the entire conversation, this memory type keeps track of a summary of the conversation.
* `ConversationSummaryBufferMemory`: merges the `ConversationSummaryMemory` and `ConversationTokenBufferMemory` types.

We'll work through each of these memory types in turn, and rewrite each one using the `RunnableWithMessageHistory` class.

## Initialize our LLM

Before jumping into our memory types, let's initialize our LLM. We're using local LLMs here with Ollama. For that we do need to pull the `llama3.2:1b-instruct-fp16` model in our terminal with:

In [1]:
#!ollama pull llama3.2:1b-instruct-fp16

Once downloaded, we initialize our LLM:

In [2]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

## 1. `ConversationBufferMemory`

`ConversationBufferMemory` is the simplest form of conversational memory, it is _literally_ just a place that we store messages, and then use to feed messages into our LLM.

Let's start with LangChain's original `ConversationBufferMemory` object, we are setting `return_messages=True` to return the messages as a list of `ChatMessage` objects — unless using a non-chat model we would always set this to `True` as without it the messages are passed as a direct string which can lead to unexpected behavior from chat LLMs.

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

C:\Users\toni\AppData\Local\Temp\ipykernel_2412\262359508.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


There are several ways that we can add messages to our memory, using the `save_context` method we can add a user query (via the `input` key) and the AI's response (via the `output` key). So, to create the following conversation:

```
User: Hi, my name is Josh
AI: Hey Josh, what's up? I'm an AI model called Zeta.
User: I'm researching the different types of conversational memory.
AI: That's interesting, what are some examples?
User: I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.
AI: That's interesting, what's the difference?
User: Buffer memory just stores the entire conversation, right?
AI: That makes sense, what about ConversationBufferWindowMemory?
User: Buffer window memory stores the last k messages, dropping the rest.
AI: Very cool!
```

We do:

In [4]:
memory.save_context(
    {"input": "Hi, my name is Josh"},  # user message
    {"output": "Hey Josh, what's up? I'm an AI model called Zeta."}  # AI response
)
memory.save_context(
    {"input": "I'm researching the different types of conversational memory."},  # user message
    {"output": "That's interesting, what are some examples?"}  # AI response
)
memory.save_context(
    {"input": "I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory."},  # user message
    {"output": "That's interesting, what's the difference?"}  # AI response
)
memory.save_context(
    {"input": "Buffer memory just stores the entire conversation, right?"},  # user message
    {"output": "That makes sense, what about ConversationBufferWindowMemory?"}  # AI response
)
memory.save_context(
    {"input": "Buffer window memory stores the last k messages, dropping the rest."},  # user message
    {"output": "Very cool!"}  # AI response
)

Before using the memory, we need to load in any variables for that memory type — in this case, there are none, so we just pass an empty dictionary:

In [5]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Josh, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessag

With that, we've created our buffer memory. Before feeding it into our LLM let's quickly view the alternative method for adding messages to our memory. With this other method, we pass individual user and AI messages via the `add_user_message` and `add_ai_message` methods. To reproduce what we did above, we do:

In [6]:
memory = ConversationBufferMemory(return_messages=True)

memory.chat_memory.add_user_message("Hi, my name is Josh")
memory.chat_memory.add_ai_message("Hey Josh, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Josh, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessag

The outcome is exactly the same in either case. To pass this onto our LLM, we need to create a `ConversationChain` object — which is already deprecated in favor of the `RunnableWithMessageHistory` class, which we will cover in a moment.

In [7]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

C:\Users\toni\AppData\Local\Temp\ipykernel_2412\2872710273.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(


In [8]:
chain.invoke({"input": "what is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is Josh', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey Josh, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kwar

{'input': 'what is my name again?',
 'history': [HumanMessage(content='Hi, my name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Josh, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, 

### `ConversationBufferMemory` with `RunnableWithMessageHistory`

As mentioned, the `ConversationBufferMemory` type is due for deprecation. Instead, we can use the `RunnableWithMessageHistory` class to implement the same functionality.

When implementing `RunnableWithMessageHistory` we will use **L**ang**C**hain **E**xpression **L**anguage (LCEL) and for this we need to define our prompt template and LLM components. Our `llm` has already been defined, so now we just define a `ChatPromptTemplate` object.

In [9]:
from langchain.prompts import (
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

We can link our `prompt_template` and our `llm` together to create a pipeline via LCEL.

In [10]:
pipeline = prompt_template | llm

Our `RunnableWithMessageHistory` requires our `pipeline` to be wrapped in a `RunnableWithMessageHistory` object. This object requires a few input parameters. One of those is `get_session_history`, which requires a function that returns a `ChatMessageHistory` object based on a session ID. We define this function ourselves:

In [11]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

We also need to tell our runnable which variable name to use for the chat history (ie `history`) and which to use for the user's query (ie `query`).

In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

Now we invoke our runnable:

In [13]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is Josh"},
    config={"session_id": "id_123"}
)

AIMessage(content="Hello Josh! How's it going? Is there anything I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:08:56.4774012Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1154725000, 'load_duration': 33653000, 'prompt_eval_count': 40, 'prompt_eval_duration': 271259600, 'eval_count': 28, 'eval_duration': 847798700, 'model_name': 'llama3.2:1b'}, id='run--3dad9c90-784a-491b-931d-adff50e44b52-0', usage_metadata={'input_tokens': 40, 'output_tokens': 28, 'total_tokens': 68})

Our chat history will now be memorized and retrieved whenever we invoke our runnable with the same session ID.

In [14]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)

AIMessage(content="Your name is Josh, which I already knew. I'm happy to chat with you, though! How's your day going so far?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:09:33.076785Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1077909900, 'load_duration': 47084800, 'prompt_eval_count': 83, 'prompt_eval_duration': 145819200, 'eval_count': 29, 'eval_duration': 881692700, 'model_name': 'llama3.2:1b'}, id='run--0c379410-9750-42ec-a001-4cfb09732592-0', usage_metadata={'input_tokens': 83, 'output_tokens': 29, 'total_tokens': 112})

We have now recreated the `ConversationBufferMemory` type using the `RunnableWithMessageHistory` class. Let's continue onto other memory types and see how these can be implemented.

## 2. `ConversationBufferWindowMemory`

The `ConversationBufferWindowMemory` type is similar to `ConversationBufferMemory`, but only keeps track of the last `k` messages. There are a few reasons why we would want to keep only the last `k` messages:

* More messages mean more tokens are sent with each request, more tokens increases latency _and_ cost.

* LLMs tend to perform worse when given more tokens, making them more likely to deviate from instructions, hallucinate, or _"forget"_ information provided to them. Conciseness is key to high performing LLMs.

* If we keep _all_ messages we will eventually hit the LLM's context window limit, by adding a window size `k` we can ensure we never hit this limit.

The buffer window solves many problems that we encounter with the standard buffer memory, while still being a very simple and intuitive form of conversational memory.

In [15]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4, return_messages=True)

C:\Users\toni\AppData\Local\Temp\ipykernel_2412\2975042918.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=4, return_messages=True)


We populate this memory using the same methods as before:

In [16]:
memory.chat_memory.add_user_message("Hi, my name is Josh")
memory.chat_memory.add_ai_message("Hey Josh, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]}

As before, we use the `ConversationChain` object (again, this is deprecated and we will rewrite it with `RunnableWithMessageHistory` in a moment).

In [17]:
chain = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

Now let's see if our LLM remembers our name:

In [18]:
chain.invoke({"input": "what is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}), AIMessage(content='That makes sense, what about 

{'input': 'what is my name again?',
 'history': [HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kw

The reason our LLM can no longer remember our name is because we have set the `k` parameter to `4`, meaning that only the last messages are stored in memory, as we can see above this does not include the first message where we introduced ourselves.

Based on the agent forgetting our name, we might wonder _why_ we would ever use this memory type compared to the standard buffer memory. Well, as with most things in AI, it is always a trade-off. Here we are able to support much longer conversations, use less tokens, and improve latency — but these come at the cost of forgetting non-recent messages.

### `ConversationBufferWindowMemory` with `RunnableWithMessageHistory`

To implement this memory type using the `RunnableWithMessageHistory` class, we can use the same approach as before. We define our `prompt_template` and `llm` as before, and then wrap our pipeline in a `RunnableWithMessageHistory` object.

For the window feature, we need to define a custom version of the `InMemoryChatMessageHistory` class that removes any messages beyond the last `k` messages.

In [19]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
        super().__init__(k=k)
        print(f"Initializing BufferWindowMessageHistory with k={k}")

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages.
        """
        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [20]:
chat_map = {}
def get_chat_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    print(f"get_chat_history called with session_id={session_id} and k={k}")
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    # remove anything beyond the last
    return chat_map[session_id]

In [21]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

Now we invoke our runnable, this time passing a `k` parameter via the `config` parameter.

In [22]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is Josh"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history called with session_id=id_k4 and k=4
Initializing BufferWindowMessageHistory with k=4


AIMessage(content="Hello Josh! How's it going? Is there anything I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:15:08.1163941Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1116673400, 'load_duration': 54702500, 'prompt_eval_count': 40, 'prompt_eval_duration': 268213100, 'eval_count': 28, 'eval_duration': 791869500, 'model_name': 'llama3.2:1b'}, id='run--2bb46e99-de9e-45f7-91aa-1655f9e4c250-0', usage_metadata={'input_tokens': 40, 'output_tokens': 28, 'total_tokens': 68})

We can also modify the messages that are stored in memory by modifying the records inside the `chat_map` dictionary directly.

In [23]:
chat_map["id_k4"].clear()  # clear the history

# manually insert history
chat_map["id_k4"].add_user_message("Hi, my name is Josh")
chat_map["id_k4"].add_ai_message("I'm an AI model called Zeta.")
chat_map["id_k4"].add_user_message("I'm researching the different types of conversational memory.")
chat_map["id_k4"].add_ai_message("That's interesting, what are some examples?")
chat_map["id_k4"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map["id_k4"].add_ai_message("That's interesting, what's the difference?")
chat_map["id_k4"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map["id_k4"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map["id_k4"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map["id_k4"].add_ai_message("Very cool!")

chat_map["id_k4"].messages

[HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]

Now let's see at which `k` value our LLM remembers our name — from the above we can already see that with `k=4` our name is not mentioned, so when running with `k=4` we should expect the LLM to forget our name:

In [24]:
pipeline_with_history.invoke(
    {"query": "what is my name again?"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history called with session_id=id_k4 and k=4


AIMessage(content="I'm not storing your name in our conversation so far. I'm just a text-based AI assistant, I don't have any information about you or your identity. Is there something else I can help you with?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:15:39.825575Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1815997400, 'load_duration': 45292300, 'prompt_eval_count': 97, 'prompt_eval_duration': 443945400, 'eval_count': 44, 'eval_duration': 1323010200, 'model_name': 'llama3.2:1b'}, id='run--7dd82a05-7deb-4c87-9534-58e809d9a2d2-0', usage_metadata={'input_tokens': 97, 'output_tokens': 44, 'total_tokens': 141})

Now let's initialize a new session with `k=14`.

In [25]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is Josh"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history called with session_id=id_k14 and k=14
Initializing BufferWindowMessageHistory with k=14


AIMessage(content="Hello Josh! How's it going? Is there anything I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:15:51.3069513Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1121167800, 'load_duration': 47619000, 'prompt_eval_count': 40, 'prompt_eval_duration': 183368600, 'eval_count': 28, 'eval_duration': 887976300, 'model_name': 'llama3.2:1b'}, id='run--08da7d9b-8092-4aa0-8c22-de94d8b57694-0', usage_metadata={'input_tokens': 40, 'output_tokens': 28, 'total_tokens': 68})

We'll manually insert the remaining messages as before:

In [26]:
chat_map["id_k14"].add_user_message("I'm researching the different types of conversational memory.")
chat_map["id_k14"].add_ai_message("That's interesting, what are some examples?")
chat_map["id_k14"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map["id_k14"].add_ai_message("That's interesting, what's the difference?")
chat_map["id_k14"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map["id_k14"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map["id_k14"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map["id_k14"].add_ai_message("Very cool!")

chat_map["id_k14"].messages

[HumanMessage(content='Hi, my name is Josh', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Josh! How's it going? Is there anything I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:15:51.3069513Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1121167800, 'load_duration': 47619000, 'prompt_eval_count': 40, 'prompt_eval_duration': 183368600, 'eval_count': 28, 'eval_duration': 887976300, 'model_name': 'llama3.2:1b'}, id='run--08da7d9b-8092-4aa0-8c22-de94d8b57694-0', usage_metadata={'input_tokens': 40, 'output_tokens': 28, 'total_tokens': 68}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I've been looking at Conversa

Now let's see if the LLM remembers our name:

In [27]:
pipeline_with_history.invoke(
    {"query": "what is my name again?"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history called with session_id=id_k14 and k=14


AIMessage(content='Your name is Josh.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:16:10.8003389Z', 'done': True, 'done_reason': 'stop', 'total_duration': 744320500, 'load_duration': 26779300, 'prompt_eval_count': 203, 'prompt_eval_duration': 560888800, 'eval_count': 6, 'eval_duration': 152096500, 'model_name': 'llama3.2:1b'}, id='run--9f3059e3-cf2b-436b-97c3-2ebc40fe9cdd-0', usage_metadata={'input_tokens': 203, 'output_tokens': 6, 'total_tokens': 209})

In [28]:
chat_map["id_k14"].messages

[HumanMessage(content='Hi, my name is Josh', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Josh! How's it going? Is there anything I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:15:51.3069513Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1121167800, 'load_duration': 47619000, 'prompt_eval_count': 40, 'prompt_eval_duration': 183368600, 'eval_count': 28, 'eval_duration': 887976300, 'model_name': 'llama3.2:1b'}, id='run--08da7d9b-8092-4aa0-8c22-de94d8b57694-0', usage_metadata={'input_tokens': 40, 'output_tokens': 28, 'total_tokens': 68}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I've been looking at Conversa

That's it! We've rewritten our buffer window memory using the recommended `RunnableWithMessageHistory` class.

## 3. `ConversationSummaryMemory`

Next up we have `ConversationSummaryMemory`, this memory type keeps track of a summary of the conversation rather than the entire conversation. This is useful for long conversations where we don't need to keep track of the entire conversation, but we do want to keep some thread of the full conversation.

As before, we'll start with the original memory class before reimplementing it with the `RunnableWithMessageHistory` class.

In [29]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)

C:\Users\toni\AppData\Local\Temp\ipykernel_2412\1675605335.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


Unlike with the previous memory types, we need to provide an `llm` to initialize `ConversationSummaryMemory`. The reason for this is that we need an LLM to generate the conversation summaries.

Beyond this small tweak, using `ConversationSummaryMemory` is the same as with our previous memory types when using the deprecated `ConversationChain` object.

In [30]:
chain = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

Let's test:

In [31]:
chain.invoke({"input": "hello there my name is Josh"})
chain.invoke({"input": "I am researching the different types of conversational memory."})
chain.invoke({"input": "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory."})
chain.invoke({"input": "Buffer memory just stores the entire conversation"})
chain.invoke({"input": "Buffer window memory stores the last k messages, dropping the rest."})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello there my name is Josh
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.

Current summary:
The human asks what the AI thinks of artificial intelligence, and the AI responds by explaining that 

{'input': 'Buffer window memory stores the last k messages, dropping the rest.',
 'history': 'The human asks about the significance of conversational memory in developing more advanced conversational AI models. The AI explains that it provides several benefits, including enabling AI systems like itself to learn from interactions and improve over time.\n\nThe human then asks what challenges are associated with leveraging conversational memory effectively. The AI discusses potential risks such as job displacement and unrealistic expectations about AI capabilities.\n\nThe human researches conversational memory again and asks for more information on how it is used in training data. The AI explains that it can draw upon previous conversations to fine-tune language understanding and generation capabilities.\n\nThe human then asks what the most significant benefit or challenge associated with conversational memory is. The AI emphasizes the importance of acknowledging limitations while working

We can see how the conversation summary varies with each new message. Let's see if the LLM is able to recall our name:

In [32]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Here is a progressively summarized version of the lines of conversation:

Current summary:
The human asks about conversational memory in developing more advanced conversational AI models.

New summary:
The human asks what benefits are associated with leveraging conversational memory effectively. The AI explains that it provides several benefits, including enabling AI systems like itself to learn from interactions and improve over time.

New lines of conversation:
Human: What happens when an AI system like yourself becomes too autonomous?
AI: As AI systems become more advanced, there's a risk that they could develop their own goals and motivations that may not align wit

{'input': 'What is my name again?',
 'history': 'Here is a progressively summarized version of the lines of conversation:\n\nCurrent summary:\nThe human asks about conversational memory in developing more advanced conversational AI models.\n\nNew summary:\nThe human asks what benefits are associated with leveraging conversational memory effectively. The AI explains that it provides several benefits, including enabling AI systems like itself to learn from interactions and improve over time.\n\nNew lines of conversation:\nHuman: What happens when an AI system like yourself becomes too autonomous?\nAI: As AI systems become more advanced, there\'s a risk that they could develop their own goals and motivations that may not align with human values. This is known as the "autonomy problem."\n\nCurrent summary:\nThe human asks about addressing the autonomy problem in AI systems.\n\nNew summary:\nThe human suggests implementing mechanisms for humans to intervene or override AI decisions in certa

As this information was stored in the summary the LLM successfully recalled our name. This may not always be the case, by summarizing the conversation we inevitably compress the full amount of information and so we may lose key details occasionally. Nonetheless, this is a great memory type for long conversations while retaining some key information.

### `ConversationSummaryMemory` with `RunnableWithMessageHistory`

Let's implement this memory type using the `RunnableWithMessageHistory` class. As with the window buffer memory, we need to define a custom implementation of the `InMemoryChatMessageHistory` class. We'll call this one `ConversationSummaryMessageHistory`.

In [33]:
from langchain_core.messages import SystemMessage


class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOllama = Field(default_factory=ChatOllama)

    def __init__(self, llm: ChatOllama):
        super().__init__(llm=llm)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages.
        """
        self.messages.extend(messages)
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible BUT keep the summary "
                "concise and no more than a short paragraph in length."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{messages}"
            )
        ])
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(existing_summary=self.messages, messages=messages)
        )
        # replace the existing history with a single system summary message 
        self.messages = [SystemMessage(content=new_summary.content)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [34]:
chat_map = {}
def get_chat_history(session_id: str, llm: ChatOllama) -> ConversationSummaryMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryMessageHistory(llm=llm)
    # return the chat history
    return chat_map[session_id]

In [35]:
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOllama,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
    ]
)

Now we invoke our runnable, this time passing a `llm` parameter via the `config` parameter.

In [36]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is Josh"},
    config={"session_id": "id_123", "llm": llm}
)

AIMessage(content="Hello Josh! How's it going? Is there anything I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:23:01.0036576Z', 'done': True, 'done_reason': 'stop', 'total_duration': 928471500, 'load_duration': 34905200, 'prompt_eval_count': 40, 'prompt_eval_duration': 136636100, 'eval_count': 28, 'eval_duration': 755318200, 'model_name': 'llama3.2:1b'}, id='run--365a7886-8566-423d-8139-562d2e7ebb43-0', usage_metadata={'input_tokens': 40, 'output_tokens': 28, 'total_tokens': 68})

Let's see what summary was generated:

In [37]:
chat_map["id_123"].messages

[SystemMessage(content="The conversation summary is: \nA user named Josh initiates a chat with an AI model. The AI responds with a greeting and asks if there's anything the user can help with or if they'd like to discuss something specific. \n\nNew messages:\n- A new message from Josh, asking for assistance or discussing a particular topic.\n- Another response from the AI model, indicating it's ready to assist.", additional_kwargs={}, response_metadata={})]

Let's continue the conversation and see if the summary is updated:

In [38]:
pipeline_with_history.invoke(
    {"query": "I'm researching the different types of conversational memory."},
    config={"session_id": "id_123", "llm": llm}
)

chat_map["id_123"].messages

[SystemMessage(content='Josh initiates a chat with an AI model, asking for assistance or discussing a specific topic. The AI responds by providing information on conversational memory, including its different types (implicit and explicit), levels of depth (surface-level, deeper-level, and long-term), and the cognitive mechanisms involved in retaining and retrieving information.', additional_kwargs={}, response_metadata={})]

So far so good! Let's continue with a few more messages before returning to the name question.

In [39]:
for msg in [
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    pipeline_with_history.invoke(
        {"query": msg},
        config={"session_id": "id_123", "llm": llm}
    )

Let's see the latest summary:

In [40]:
chat_map["id_123"].messages

[SystemMessage(content='Conversational buffer memory typically stores a limited number of recent messages, often around 5-10, depending on the chatbot\'s configuration and usage patterns. This is usually referred to as the "buffer window" or "recent history". The idea is that by storing only the most recent messages, the chatbot can quickly retrieve information from its conversational memory when needed.', additional_kwargs={}, response_metadata={})]

The information about our name has been maintained, so let's see if this is enough for our LLM to correctly recall our name.

In [41]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123", "llm": llm}
)

AIMessage(content="I'm happy to help you with your question! However, I don't have any record of a previous conversation or interaction with you. This is our first time chatting, and I'm starting from scratch. My name is Zeta, by the way. How can I assist you today?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:25:55.5305681Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2112774900, 'load_duration': 44438700, 'prompt_eval_count': 115, 'prompt_eval_duration': 446350300, 'eval_count': 59, 'eval_duration': 1620606300, 'model_name': 'llama3.2:1b'}, id='run--a4a12d95-2bcf-4ef8-a3a8-8370405d31ad-0', usage_metadata={'input_tokens': 115, 'output_tokens': 59, 'total_tokens': 174})

Perfect! We've successfully implemented the `ConversationSummaryMemory` type using the `RunnableWithMessageHistory` class.

## 4. `ConversationSummaryBufferMemory`

Our final memory type acts as a combination of `ConversationSummaryMemory` and `ConversationBufferMemory`. It keeps the buffer for the conversation up until the previous `n` tokens, anything beyond that limit is summarized then dropped from the buffer. Producing something like:


```
# ~~ a summary of previous interactions
The user named Josh introduced himself and the AI responded, introducing itself as an AI model called Zeta.
Josh then said he was researching the different types of conversational memory and Zeta asked for some
examples.
# ~~ the most recent messages
Human: I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.
AI: That's interesting, what's the difference?
Human: Buffer memory just stores the entire conversation
AI: That makes sense, what about ConversationBufferWindowMemory?
Human: Buffer window memory stores the last k messages, dropping the rest.
AI: Very cool!
```


In [42]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    return_messages=True
)

C:\Users\toni\AppData\Local\Temp\ipykernel_2412\789717621.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


As before, we set up the deprecated memory type using the `ConversationChain` object.

In [43]:
chain = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

First invoke with a single message:

In [44]:
chain.invoke({"input": "Hi, my name is Josh"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi, my name is Josh
AI:


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\toni\engineering\ml\langchain\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\toni\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]


> Finished chain.


{'input': 'Hi, my name is Josh',
 'history': [HumanMessage(content='Hi, my name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm happy to chat with you, Josh. My name is Nova, by the way. I'm an AI designed to assist and communicate with humans in a helpful and informative way. How's your day going so far?", additional_kwargs={}, response_metadata={})],
 'response': "I'm happy to chat with you, Josh. My name is Nova, by the way. I'm an AI designed to assist and communicate with humans in a helpful and informative way. How's your day going so far?"}

---

**! If you see _PermissionError_ you need to modify folder permissions _or_ switch where your models/datasets/etc are being saved/loaded from — if the error shows a path like `~/.cache/huggingface/hub` you can run `sudo chmod 777 ~/.cache/huggingface/hub` in your terminal window to update permissions.**

---

Looks good so far, let's continue with a few more messages:

In [45]:
for i, msg in enumerate([
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]):
    print(f"---\nMessage {i+1}\n---\n")
    chain.invoke({"input": msg})

---
Message 1
---



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is Josh', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm happy to chat with you, Josh. My name is Nova, by the way. I'm an AI designed to assist and communicate with humans in a helpful and informative way. How's your day going so far?", additional_kwargs={}, response_metadata={})]
Human: I'm researching the different types of conversational memory.
AI:

> Finished chain.
---
Message 2
---



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its cont

We can see with each new message the initial `SystemMessage` is updated with a new summary of the conversation. This initial `SystemMessage` is then followed by the most recent `AIMessage` and `HumanMessage` objects.

### `ConversationSummaryBufferMemory` with `RunnableWithMessageHistory`

As with the previous memory types, we will implement this memory type again using the `RunnableWithMessageHistory` class. In our implementation we will modify the buffer window to be based on the number of messages rather than number of tokens. This tweak will make our implementation more closely aligned with original buffer window.

We will implement all of this via a new `ConversationSummaryBufferMessageHistory` class.

In [46]:
class ConversationSummaryBufferMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOllama = Field(default_factory=ChatOllama)
    k: int = Field(default_factory=int)

    def __init__(self, llm: ChatOllama, k: int):
        super().__init__(llm=llm, k=k)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages and summarizing the messages that we
        drop.
        """
        existing_summary = None
        old_messages = None
        # see if we already have a summary message
        if len(self.messages) > 0 and isinstance(self.messages[0], SystemMessage):
            print(">> Found existing summary")
            existing_summary: str | None = self.messages.pop(0)
        # add the new messages to the history
        self.messages.extend(messages)
        # check if we have too many messages
        if len(self.messages) > self.k:
            print(
                f">> Found {len(self.messages)} messages, dropping "
                f"latest {len(self.messages) - self.k} messages.")
            # pull out the oldest messages...
            old_messages = self.messages[:self.k]
            # ...and keep only the most recent messages
            self.messages = self.messages[-self.k:]
        if old_messages is None:
            print(">> No old messages to update summary with")
            # if we have no old_messages, we have nothing to update in summary
            return
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible BUT keep the summary "
                "concise and no more than a short paragraph in length."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{old_messages}"
            )
        ])
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=existing_summary,
                old_messages=old_messages
            )
        )
        print(f">> New summary: {new_summary.content}")
        # prepend the new summary to the history
        self.messages = [SystemMessage(content=new_summary.content)] + self.messages

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

Redefine the `get_chat_history` function to use our new `ConversationSummaryBufferMessageHistory` class.

In [47]:
chat_map = {}
def get_chat_history(
    session_id: str,
    llm: ChatOllama,
    k: int
) -> ConversationSummaryBufferMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryBufferMessageHistory(llm=llm, k=k)
    # return the chat history
    return chat_map[session_id]

Setup our pipeline with new configurable fields.

In [48]:
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOllama,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

Finally, we invoke our runnable:

In [49]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is Josh"},
    config={"session_id": "id_123", "llm": llm, "k": 4}
)
chat_map["id_123"].messages

>> No old messages to update summary with


[HumanMessage(content='Hi, my name is Josh', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Josh! How's it going? Is there anything I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-12T10:33:46.0011203Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1040205100, 'load_duration': 47060800, 'prompt_eval_count': 40, 'prompt_eval_duration': 200227000, 'eval_count': 28, 'eval_duration': 790577200, 'model_name': 'llama3.2:1b'}, id='run--81ca5140-eeb1-4796-a40e-4460bba6b17e-0', usage_metadata={'input_tokens': 40, 'output_tokens': 28, 'total_tokens': 68})]

In [50]:
for i, msg in enumerate([
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]):
    print(f"---\nMessage {i+1}\n---\n")
    pipeline_with_history.invoke(
        {"query": msg},
        config={"session_id": "id_123", "llm": llm, "k": 4}
    )

---
Message 1
---

>> No old messages to update summary with
---
Message 2
---

>> Found 6 messages, dropping latest 2 messages.
>> New summary: Josh is researching conversational memory, which refers to the ability to retain and recall information from previous conversations. Conversational memory involves recalling previously discussed topics or ideas, capturing context, storing factual information, and understanding nuances of language and context. Researchers identify factors such as repetition, contextual cues, and emotional connection that contribute to conversational memory.
---
Message 3
---

>> Found existing summary
>> Found 6 messages, dropping latest 2 messages.
>> New summary: conversational memory refers to the ability to retain and recall information from previous conversations, allowing us to pick up where we left off and continue a discussion without needing to repeat ourselves. Researchers identify factors such as repetition, contextual cues, and emotional connection 

There we go, we've successfully implemented the `ConversationSummaryBufferMemory` type using `RunnableWithMessageHistory`!

---